In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [40]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=3,
    lora_alpha=9,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 704,260 || all params: 125,352,968 || trainable%: 0.5618


In [41]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [42]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.369500,1.325122,0.776563
200,0.983900,0.409690,0.871875
300,0.384500,0.341478,0.881250
400,0.305300,0.323629,0.893750
500,0.315200,0.318294,0.900000
600,0.304400,0.322800,0.900000
700,0.318800,0.307365,0.903125
800,0.317000,0.300685,0.906250
900,0.303800,0.304718,0.910937
1000,0.285500,0.302558,0.906250


In [44]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-900'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:58<00:00,  4.29it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-900.csv


In [45]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3695,1.325122,0.350625,0.776563,0.000048,0.013405,0.044378,1.033490,-0.425938,0.451509
1,200,0.9839,0.409690,0.766071,0.871875,0.000045,0.026810,0.574210,2.401571,-0.105804,0.878648
2,300,0.3845,0.341478,0.873661,0.881250,0.000043,0.040214,0.043022,1.125987,-0.007589,0.991388
3,400,0.3053,0.323629,0.896429,0.893750,0.000040,0.053619,-0.018329,0.943364,0.002679,1.002997
4,500,0.3152,0.318294,0.890179,0.900000,0.000037,0.067024,-0.003094,0.990281,-0.009821,0.989087
5,600,0.3044,0.322800,0.897768,0.900000,0.000035,0.080429,-0.018400,0.942999,-0.002232,0.997520
6,700,0.3188,0.307365,0.896875,0.903125,0.000033,0.093834,0.011435,1.037204,-0.006250,0.993080
7,800,0.3170,0.300685,0.891964,0.906250,0.000030,0.107239,0.016315,1.054260,-0.014286,0.984236
8,900,0.3038,0.304718,0.899554,0.910937,0.000028,0.120643,-0.000918,0.996987,-0.011384,0.987503
9,1000,0.2855,0.302558,0.906696,0.906250,0.000025,0.134048,-0.017058,0.943621,0.000446,1.000493
